# World Cup 2022 Prediction

## Introduction

With the World Cup ongoing at the time of writing this, it will be very interesting to predict the winners. In this notebook, we used past international game results to learn, cross validate, and test a linear regression model that will predict the expected goals for each match. This dataset can be found in Past_International_Matches.csv, which we derived from Kaggle ([FIFA World Cup 2022 ⚽️🏆](https://www.kaggle.com/datasets/brenda89/fifa-world-cup-2022)).

We then used this model then simulate the whole competion. This means predicting each game in the group stage to find out who will qualify for the knock out stages, and then use the predefined tournament bracket for this World Cup. We will iterate many simulations to find the chance of each team winning this World Cup.

## Import and Set Up

In this section we will import the necessary libraries, read the dataset, and store it in matrices.

In [4]:
import pandas as pd

In [8]:
groups = pd.read_csv('Fifa_Worldcup_2022_Groups_Final.csv')
data = pd.read_csv('Past_International_Matches.csv')
data = data.set_index("Index")

,Group,Country_Name,First,Match_index,Second,Match_index.1,Third,Match_index.2
0,A1,Qatar,Ecuador,1,Senegal,17,Netherlands,33
1,A2,Ecuador,Qatar,1,Netherlands,18,Senegal,34
2,A3,Senegal,Netherlands,2,Qatar,17,Ecuador,34
3,A4,Netherlands,Senegal,2,Ecuador,18,Qatar,33
4,B1,England,IR Iran,3,USA,19,Wales,35
5,B2,IR Iran,England,3,Wales,20,USA,36
6,B3,USA,Wales,4,England,19,IR Iran,36
7,B4,Wales,USA,4,IR Iran,20,England,35
8,C1,Argentina,Saudi Arabia,5,Mexico,21,Poland,37
9,C2,Saudi Arabia,Argentina,5,Poland,22,Mexico,38


## Training 

In [ ]:
x,y = data.loc(:, [])